In [7]:
import sys
principal_path = '../'
if principal_path not in sys.path:
    sys.path.append('../')

In [8]:
import os
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import MNIST
import pytorch_lightning as pl
import torchmetrics
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from models.MNISTModel import MNISTModel
from models.MNISTModelWithBottelNeck import MNISTModelWithBottelNeck

In [9]:
SEED = 2024
pl.seed_everything(SEED)

Seed set to 2024


2024

In [10]:
LEARING_RATE = 1e-3
BATCH_SIZE = 64
EPOCHS = 5
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {DEVICE}')

Using cuda


## Download Dataset

In [11]:
def load_data(batch_size=BATCH_SIZE, num_workers=4):
    # Transformaciones para los datos
    transform = transforms.ToTensor()

    # Carga de datos de entrenamiento
    mnist_train = MNIST(os.getcwd(), train=True, download=True, transform=transform)
    
    # División entre entrenamiento y validación
    train_size = int(0.8 * len(mnist_train))
    val_size = len(mnist_train) - train_size
    mnist_train, mnist_val = random_split(mnist_train, [train_size, val_size])

    # DataLoader para entrenamiento y validación
    train_loader = DataLoader(mnist_train, batch_size=batch_size, num_workers=num_workers, shuffle=True, persistent_workers=True)
    val_loader = DataLoader(mnist_val, batch_size=batch_size, num_workers=num_workers, shuffle=False, persistent_workers=True)

    # Carga de datos de test
    mnist_test = MNIST(os.getcwd(), train=False, download=True, transform=transform)
    test_loader = DataLoader(mnist_test, batch_size=batch_size, num_workers=num_workers, persistent_workers=True)

    return train_loader, val_loader, test_loader

train_loader, val_loader, test_loader = load_data()

## Training

### MNISTmodel

In [12]:
model = MNISTModel(lr=LEARING_RATE)

In [13]:
trainer = pl.Trainer(max_epochs=EPOCHS)
trainer.fit(model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\code\ExAI\exai\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html

Epoch 4: 100%|██████████| 750/750 [00:03<00:00, 220.19it/s, v_num=3, train_loss=0.040, train_acc=0.984, val_loss=0.0966, val_acc=0.972] 

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 750/750 [00:03<00:00, 219.74it/s, v_num=3, train_loss=0.040, train_acc=0.984, val_loss=0.0966, val_acc=0.972]


In [14]:
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 157/157 [00:00<00:00, 199.05it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
           f1               0.9711270332336426
        precision           0.9746068120002747
         recall             0.9731449484825134
        test_acc            0.9749000072479248
        test_loss            0.086024709045887
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.086024709045887,
  'test_acc': 0.9749000072479248,
  'precision': 0.9746068120002747,
  'recall': 0.9731449484825134,
  'f1': 0.9711270332336426}]

### Bottleneck

In [15]:
bottel_neck_model = MNISTModelWithBottelNeck(lr=LEARING_RATE)

In [16]:
trainer = pl.Trainer(max_epochs=EPOCHS)
trainer.fit(bottel_neck_model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                | Params
--------------------------------------------------
0 | train_acc | MulticlassAccuracy  | 0     
1 | val_acc   | MulticlassAccuracy  | 0     
2 | test_acc  | MulticlassAccuracy  | 0     
3 | precision | MulticlassPrecision | 0     
4 | recall    | MulticlassRecall    | 0     
5 | f1        | MulticlassF1Score   | 0     
6 | layer_1   | Linear              | 200 K 
7 | layer_2   | Linear              | 32.9 K
8 | layer_3   | Linear              | 33.0 K
9 | layer_4   | Linear              | 2.6 K 
--------------------------------------------------
269 K     Trainable params
0         Non-trainable params
269 K     Total params
1.078     Total estimated model params size (MB)


Epoch 4: 100%|██████████| 750/750 [00:03<00:00, 226.20it/s, v_num=4, train_loss=0.098, train_acc=0.953, val_loss=0.128, val_acc=0.961]  

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 750/750 [00:03<00:00, 225.48it/s, v_num=4, train_loss=0.098, train_acc=0.953, val_loss=0.128, val_acc=0.961]


In [17]:
trainer.test(bottel_neck_model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 157/157 [00:00<00:00, 210.53it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
           f1               0.9615684747695923
        precision           0.9653060436248779
         recall             0.9644773602485657
        test_acc            0.9660999774932861
        test_loss           0.11590377241373062
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.11590377241373062,
  'test_acc': 0.9660999774932861,
  'precision': 0.9653060436248779,
  'recall': 0.9644773602485657,
  'f1': 0.9615684747695923}]